In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!ln -s /content/drive/My\ Drive/ /my\drive

ln: failed to create symbolic link '/mydrive/My Drive': File exists


In [3]:
#파이토치 불러오기
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [4]:

!pip install "pillow<6.2.2"

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [6]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')

In [27]:
EPOCHS = 30
BATCH_SIZE = 64

In [28]:
transform = transforms.Compose([transforms.ToTensor()])

In [29]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    download = True, 
    transform = transform
)

testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    download = True,
    transform = transform   
)

train_loader = torch.utils.data.DataLoader(
    dataset = trainset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

test_loader = torch.utils.data.DataLoader(
    dataset = testset,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

In [30]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(784, 256) # fully connected(Dense), 784개 입력 받아 256개 출력
    self.fc2 = nn.Linear(256,128) # 128개 입력받아 10개 출력 (패션아이템 카테고리 겟수)
    self.fc3 = nn.Linear(128,10)

  def forward(self, x):
    x = x.view(-1, 784) #입력을 받아 1차원 행렬로 만듦
    x = F.relu(self.fc1(x)) #nn.ReLU 클래스와 같은 기능
    x = F.relu(self.fc2(x))
    x = self.fc3(x) #출력값 10개
    return x

In [31]:
model = Net().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [32]:
def train(model, train_loader, optimizer):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):  
   
    # 학습 데이터를 DEVICE의 메모리로 보냄
    data, target = data.to(DEVICE), target.to(DEVICE)  # to() 함수로 모델 가중치에 사용한 같은 장치이용(CPU or GPU)

    optimizer.zero_grad()
    output= model(data)
    loss = F.cross_entropy(output, target) # 예측값(output)과 레이블(target)값 오차 구하기, 교차엔트로피

    loss.backward()  # 기울기 계산
    optimizer.step() # 계산한 기울기를 모델의 맞춰 가중치 수정 


In [33]:
def evaluate(model, test_loader):
  model.eval()
  test_loss = 0
  correct = 0 
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output = model(data)

      # 모든 오차 더하기, 미니배치 평균이 아닌 합 ie. test_loss는 모든 테스트셋의 오차 합이 됨
      test_loss += F.cross_entropy(output, target, reduction='sum').item()

      # 가장 큰 값을 가진 클래스가 모델의 에측. 예측과 정답을 비교하여 일치할 경우 correct에 1을 더함
      pred = output.max(1, keepdim=True)[1]
      correct += pred.eq(target.view_as(pred)).sum().item() # sun90으로 배치에서 모델이 정답을 맞춘 갯수구함

  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy


In [34]:
for epoch in range (1, EPOCHS + 1):
  train(model, train_loader, optimizer)
  test_loss, test_accuracy = evaluate(model, test_loader)

  print('[{}], Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(
    epoch, test_loss, test_accuracy)) 

[1], Test Loss: 0.8285, Accuracy: 68.54%
[2], Test Loss: 0.7103, Accuracy: 72.83%
[3], Test Loss: 0.5904, Accuracy: 78.83%
[4], Test Loss: 0.5456, Accuracy: 80.93%
[5], Test Loss: 0.5299, Accuracy: 81.01%
[6], Test Loss: 0.5045, Accuracy: 82.17%
[7], Test Loss: 0.4799, Accuracy: 82.83%
[8], Test Loss: 0.4694, Accuracy: 83.67%
[9], Test Loss: 0.5022, Accuracy: 82.18%
[10], Test Loss: 0.4596, Accuracy: 83.58%
[11], Test Loss: 0.4688, Accuracy: 83.24%
[12], Test Loss: 0.4564, Accuracy: 84.02%
[13], Test Loss: 0.4383, Accuracy: 84.25%
[14], Test Loss: 0.4354, Accuracy: 84.53%
[15], Test Loss: 0.4261, Accuracy: 84.87%
[16], Test Loss: 0.4172, Accuracy: 85.34%
[17], Test Loss: 0.4150, Accuracy: 85.49%
[18], Test Loss: 0.4164, Accuracy: 85.14%
[19], Test Loss: 0.4008, Accuracy: 85.65%
[20], Test Loss: 0.4244, Accuracy: 85.11%
[21], Test Loss: 0.3995, Accuracy: 86.05%
[22], Test Loss: 0.4468, Accuracy: 83.42%
[23], Test Loss: 0.3889, Accuracy: 86.22%
[24], Test Loss: 0.3846, Accuracy: 86.48%
[